In [7]:
## Optional
# Format output of Jupyter Notebook
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

# Hide python warnings
import warnings
warnings.filterwarnings('ignore')

# Initialize Spark Session with configs

In [4]:
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType
from pyspark.sql.functions import lit, isnull, when, count, col, regexp_extract, concat_ws, to_date, expr, quarter, when, date_add, year, month, day, dayofweek, broadcast, avg, min, max, like

# TODO - parameterize endpoints/ports such as minio:9000 with envs

# Define config for SparkSession,
# such as Iceberg catalog that utilizes minio, an S3-compatible local object storage
spark_configs = {
    'spark.master': 'spark://spark-iceberg:7077',
    'spark.sql.catalog.airline': 'org.apache.iceberg.spark.SparkCatalog',
    # 'spark.sql.catalog.airline': 'org.apache.iceberg.spark.SparkSessionCatalog',
    'spark.sql.catalog.airline.io-impl': 'org.apache.iceberg.aws.s3.S3FileIO',
    'spark.sql.catalog.airline.s3.endpoint': 'http://minio:9000',
    'spark.sql.catalog.airline.type': 'rest',
    'spark.sql.catalog.airline.uri': 'http://rest:8181',
    'spark.sql.catalog.airline.warehouse': 's3://warehouse',
    'spark.sql.defaultCatalog': 'airline',
    'spark.executor.memory': '2g'
}

# Initialize SparkSession
spark = (
    SparkSession
    .builder
    .appName('EDA')
    .config(map=spark_configs)
    .getOrCreate()
)

In [5]:
spark.conf.get('spark.executor.memory')

'2g'

In [7]:
import pprint as pp

# print(spark.sparkContext.uiWebUrl)

print(spark.conf.get('spark.app.id'))
print(spark.conf.get('spark.app.name'))
print(spark.conf.get('spark.app.startTime'))
print(spark.conf.get('spark.master'))
print()
print(spark.conf.get('spark.sql.catalog.airline'))
print(spark.conf.get('spark.sql.catalog.airline.io-impl'))
print(spark.conf.get('spark.sql.catalog.airline.s3.endpoint'))
print(spark.conf.get('spark.sql.catalog.airline.type'))
print(spark.conf.get('spark.sql.catalog.airline.uri'))
print(spark.conf.get('spark.sql.catalog.airline.warehouse'))
print()

                     
# pp.pp(dict(sc.getConf().getAll()), sort_dicts=True, )

local-1734722159949
EDA
1734722158776
spark://spark-iceberg:7077

org.apache.iceberg.spark.SparkCatalog
org.apache.iceberg.aws.s3.S3FileIO
http://minio:9000
rest
http://rest:8181
s3://warehouse



# Ingest raw csv data

### Read flights.csv and perform transforms

In [2]:
# Ingest flights data

# TODO: Fix nullables column, cant set column as Not-Nullable for some reason...

# Define ingest schema
flights_schema = StructType([
    StructField("year", IntegerType(), False),   # Non-Nullable
    StructField("month", IntegerType(), False),  # Non-Nullable
    StructField("day", IntegerType(), False),    # Non-Nullable
    StructField("day_of_week", IntegerType(), False),    # Non-Nullable
    StructField("airline", StringType(), False),         # Non-Nullable
    StructField("flight_number", IntegerType(), False),  # Non-Nullable
    StructField("tail_number", StringType(), True),
    StructField("origin_airport", StringType(), False),  # Non-Nullable
    StructField("destination_airport", StringType(), False),   # Non-Nullable
    StructField("scheduled_departure", IntegerType(), False),  # Non-Nullable
    StructField("departure_time", IntegerType(), True),
    StructField("departure_delay", IntegerType(), True),
    StructField("taxi_out", IntegerType(), True),
    StructField("wheels_off", IntegerType(), True),
    StructField("scheduled_time", IntegerType(), True),
    StructField("elapsed_time", IntegerType(), True),
    StructField("air_time", IntegerType(), True),
    StructField("distance", IntegerType(), False),  # Non-Nullable
    StructField("wheels_on", IntegerType(), True),
    StructField("taxi_in", IntegerType(), True),
    StructField("scheduled_arrival", IntegerType(), False),  # Non-Nullable
    StructField("arrival_time", IntegerType(), True),
    StructField("arrival_delay", IntegerType(), True),
    StructField("diverted", IntegerType(), False),   # Non-Nullable
    StructField("cancelled", IntegerType(), False),  # Non-Nullable
    StructField("cancellation_reason", StringType(), True),
    StructField("air_system_delay", IntegerType(), True),
    StructField("security_delay", IntegerType(), True),
    StructField("airline_delay", IntegerType(), True),
    StructField("late_aircraft_delay", IntegerType(), True),
    StructField("weather_delay", IntegerType(), True)
])


# Read flights.csv
filename = '/home/iceberg/data/flights.csv'
flights_df = spark.read.schema(flights_schema).csv(
    '/home/iceberg/data/flights.csv',
    schema=flights_schema,
    enforceSchema=True,
    header = True
)

# Add date column and remove other date-related columns
flights_df = flights_df \
    .withColumn('date', to_date(concat_ws('-', 'year', 'month', 'day'))) \
    .drop('year', 'month', 'day', 'day_of_week')

# Add is_delayed column (when scheduled_departure > 0)
flights_df = flights_df \
    .withColumn(
        'is_delayed',
        when(col('departure_delay') > 0, lit(1)).otherwise(lit(0))
    )

# Rearrange date to be first column,
flights_df = flights_df.select('date', *[col(c) for c in flights_df.columns if c != 'date'])

# Sort by date and schedule_departure time
flights_df = flights_df.sort(['date', 'scheduled_departure'])

# flights_df.cache()
# flights_df.printSchema()
flights_df.show()
flights_df.explain()

24/12/27 23:57:17 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+----------+
|      date|airline|flight_number|tail_number|origin_airport|destination_airport|scheduled_departure|departure_time|departure_delay|taxi_out|wheels_off|scheduled_time|elapsed_time|air_time|distance|wheels_on|taxi_in|scheduled_arrival|arrival_time|arrival_delay|diverted|cancelled|cancellation_reason|air_system_delay|security_delay|airline_delay|late_aircraft_delay|weather_delay|is_delayed|
+----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+

In [ ]:
# Experimenting

# flights_df.groupBy(['flight_number', 'tail_number']).count().show()
# flights_df.filter(col('flight_number') == 2519).sort(['airline', 'month', 'day']).show(50)

# # Spark SQL
# date_df = flights_df.select(['year', 'month', 'day', 'day_of_week']).withColumn('date', expr('to_date(concat(year, "-", month))'))

### Create dim_date table

In [2]:
# Original day_of_week 1 is Monday, spark dayofweek 1 is Sunday
date_df = spark.range(365) \
    .withColumn('date', expr('date_add("2015-01-01", CAST(id AS INT))')) \
    .withColumn('year', year('date')) \
    .withColumn('month', month('date')) \
    .withColumn('day', day('date')) \
    .withColumn('day_of_week', dayofweek('date')) \
    .withColumn('quarter', quarter('date')) \
    .drop('id')

# List of U.S. federal holidays
us_holidays_2015 = [
    ("2015-01-01", "New Year's Day"),
    ("2015-01-19", "Martin Luther King Jr. Day"),
    ("2015-02-16", "Presidents' Day"),
    ("2015-05-25", "Memorial Day"),
    ("2015-07-04", "Independence Day"),
    ("2015-09-07", "Labor Day"),
    ("2015-10-12", "Columbus Day"),
    ("2015-11-11", "Veterans Day"),
    ("2015-11-26", "Thanksgiving Day"),
    ("2015-12-25", "Christmas Day"),
]

# Create holidays_df and cast date from STRING to DATE type
holidays_df = spark.createDataFrame(us_holidays_2015, ['holiday_date', 'holiday_name'])
holidays_df = holidays_df.withColumn('holiday_date', to_date('holiday_date'))

# Join holidays to date_df and add is_holiday column
date_df = date_df \
    .join(
        broadcast(holidays_df),
        date_df.date == holidays_df.holiday_date,
        'left'
    ) \
    .withColumn(
        'is_holiday',
        when(col('holiday_name').isNotNull(), lit(True)).otherwise(lit(False))
    ) \
    .drop('holiday_date') \
    .sort('date')

# Rearrange date to be first column,
date_df = date_df.select('date', *[col(c) for c in date_df.columns if c != 'date'])

date_df.show()
date_df.explain()

+----------+----+-----+---+-----------+-------+--------------------+----------+
|      date|year|month|day|day_of_week|quarter|        holiday_name|is_holiday|
+----------+----+-----+---+-----------+-------+--------------------+----------+
|2015-01-01|2015|    1|  1|          5|      1|      New Year's Day|      true|
|2015-01-02|2015|    1|  2|          6|      1|                NULL|     false|
|2015-01-03|2015|    1|  3|          7|      1|                NULL|     false|
|2015-01-04|2015|    1|  4|          1|      1|                NULL|     false|
|2015-01-05|2015|    1|  5|          2|      1|                NULL|     false|
|2015-01-06|2015|    1|  6|          3|      1|                NULL|     false|
|2015-01-07|2015|    1|  7|          4|      1|                NULL|     false|
|2015-01-08|2015|    1|  8|          5|      1|                NULL|     false|
|2015-01-09|2015|    1|  9|          6|      1|                NULL|     false|
|2015-01-10|2015|    1| 10|          7| 

# Explore data!

TODO:
- Check statistics of important columns, like what are the percentiles for departure/arrival delay?
- Check max values for delay

## Total number of flight records: 5,819,079

In [32]:
flights_df.count()

5819079

## Total number of cancelled flights: 89,884
These will cause nulls in other columns.

In [184]:
cancelled_df = flights_df.where('cancelled = 1')
cancelled_df.count()
# total_df.show()

# Find flights that were taxied out or took off, but were ultimately cancelled
cancelled_df.filter(col('departure_delay').isNotNull()).show()

+----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+----------+----+-----+---+-----------+-------+------------+----------+
|      date|airline|flight_number|tail_number|origin_airport|destination_airport|scheduled_departure|departure_time|departure_delay|taxi_out|wheels_off|scheduled_time|elapsed_time|air_time|distance|wheels_on|taxi_in|scheduled_arrival|arrival_time|arrival_delay|diverted|cancelled|cancellation_reason|air_system_delay|security_delay|airline_delay|late_aircraft_delay|weather_delay|is_delayed|year|month|day|day_of_week|quarter|holiday_name|is_holiday|
+----------+-------+-------------+-----------+--------------+-------------------+-----------------

In [196]:
# Confirm cancellation reason is not null only when cancelled == 1
flights_df.filter(
    (col('cancelled') == 0) & (col('cancellation_reason').isNotNull())
).show()

+----+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+----------+----+-----+---+-----------+-------+------------+----------+
|date|airline|flight_number|tail_number|origin_airport|destination_airport|scheduled_departure|departure_time|departure_delay|taxi_out|wheels_off|scheduled_time|elapsed_time|air_time|distance|wheels_on|taxi_in|scheduled_arrival|arrival_time|arrival_delay|diverted|cancelled|cancellation_reason|air_system_delay|security_delay|airline_delay|late_aircraft_delay|weather_delay|is_delayed|year|month|day|day_of_week|quarter|holiday_name|is_holiday|
+----+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+

## Total number of diverted flights: 15,187

In [189]:
diverted_df = flights_df.where('diverted = 1')
print(diverted_df.count())
diverted_df.show()
diverted_df.filter(col('arrival_delay').isNotNull()).show()

15187
+----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+----------+----+-----+---+-----------+-------+------------+----------+
|      date|airline|flight_number|tail_number|origin_airport|destination_airport|scheduled_departure|departure_time|departure_delay|taxi_out|wheels_off|scheduled_time|elapsed_time|air_time|distance|wheels_on|taxi_in|scheduled_arrival|arrival_time|arrival_delay|diverted|cancelled|cancellation_reason|air_system_delay|security_delay|airline_delay|late_aircraft_delay|weather_delay|is_delayed|year|month|day|day_of_week|quarter|holiday_name|is_holiday|
+----------+-------+-------------+-----------+--------------+-------------------+-----------

## Get Null counts for each column

In [31]:
flights_df.select([count(when(isnull(c), c)).alias(c) for c in data.columns]).show()

+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+-

## Investigate some causes for nulls in each column

In [48]:
# List of columns with null values
null_columns = [col for col in flights_df.columns if flights_df.filter(f"{col} IS NULL").count() > 0]

# Iterate through columns to investigate null causes
for col in null_columns:
    print(f"Investigating nulls in column: {col}")
    # Count rows where column is null and cancelled == 1
    count = flights_df.where(f"`{col}` IS NULL AND cancelled = 1").count()
    print(f"Nulls in {col} where flight is cancelled: {count}\n")

Investigating nulls in column: tail_number


Nulls in tail_number where flight is cancelled: 14721

Investigating nulls in column: departure_time


Nulls in departure_time where flight is cancelled: 86153

Investigating nulls in column: departure_delay


Nulls in departure_delay where flight is cancelled: 86153

Investigating nulls in column: taxi_out


Nulls in taxi_out where flight is cancelled: 89047

Investigating nulls in column: wheels_off


...

Nulls in air_system_delay where flight is cancelled: 89884

Investigating nulls in column: security_delay


Nulls in security_delay where flight is cancelled: 89884

Investigating nulls in column: airline_delay


Nulls in airline_delay where flight is cancelled: 89884

Investigating nulls in column: late_aircraft_delay


Nulls in late_aircraft_delay where flight is cancelled: 89884

Investigating nulls in column: weather_delay


Nulls in weather_delay where flight is cancelled: 89884



#### Tail_number - all Nulls are from cancelled flights
Although, not all cancelled flights have Nulls in tail_number. Possibly some flights are cancelled before an aircraft is assigned to the flight.

In [40]:
flights_df.where('tail_number IS NULL AND cancelled = 1').count()

14721

In [44]:
flights_df.where('departure_time IS NULL and cancelled = 1').count()

86153

#### Scheduled_time - 6 Nulls, 5 out of 6 flight were cancelled, 1 flight was diverted
These flights took off, but did not arrive in destination.
Scheduled_time is the estimated time in air?

In [28]:
flights_df.where('scheduled_time IS NULL').show()

+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
|year|month|day|day_of_week|airline|flight_number|tail_number|origin_airport|destination_airport|scheduled_departure|departure_time|departure_delay|taxi_out|wheels_off|scheduled_time|elapsed_time|air_time|distance|wheels_on|taxi_in|scheduled_arrival|arrival_time|arrival_delay|diverted|cancelled|cancellation_reason|air_system_delay|security_delay|airline_delay|late_aircraft_delay|weather_delay|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+-

In [63]:
# Identify primary keys. Unique identifier is date, airline, flight_number
flights_df = spark.table('airline.db.flights')
flights_df.filter(
    """
    date='2015-01-01' AND flight_number=7 AND airline='WN'
    """
).show()
flights_df.groupBy('date', 'flight_number', 'airline').count().sort('date', 'flight_number', 'airline').show(40)

+----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+----------+
|      date|airline|flight_number|tail_number|origin_airport|destination_airport|scheduled_departure|departure_time|departure_delay|taxi_out|wheels_off|scheduled_time|elapsed_time|air_time|distance|wheels_on|taxi_in|scheduled_arrival|arrival_time|arrival_delay|diverted|cancelled|cancellation_reason|air_system_delay|security_delay|airline_delay|late_aircraft_delay|weather_delay|is_delayed|
+----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+

24/12/28 04:33:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/28 04:33:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/28 04:33:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/28 04:33:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/28 04:33:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/28 04:33:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/28 04:33:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/28 04:33:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/28 04:33:35 WARN RowBasedKeyValueBatch: Calling spill() on

+----------+-------------+-------+-----+
|      date|flight_number|airline|count|
+----------+-------------+-------+-----+
|2015-01-01|            1|     AA|    1|
|2015-01-01|            1|     AS|    1|
|2015-01-01|            1|     B6|    1|
|2015-01-01|            1|     HA|    1|
|2015-01-01|            1|     VX|    1|
|2015-01-01|            2|     AA|    1|
|2015-01-01|            2|     AS|    1|
|2015-01-01|            2|     B6|    1|
|2015-01-01|            2|     HA|    1|
|2015-01-01|            2|     VX|    1|
|2015-01-01|            3|     AA|    1|
|2015-01-01|            3|     AS|    1|
|2015-01-01|            3|     B6|    1|
|2015-01-01|            3|     HA|    1|
|2015-01-01|            3|     UA|    1|
|2015-01-01|            4|     AA|    1|
|2015-01-01|            4|     AS|    1|
|2015-01-01|            4|     B6|    1|
|2015-01-01|            4|     HA|    1|
|2015-01-01|            5|     AA|    1|
|2015-01-01|            5|     AS|    1|
|2015-01-01|    

# Write to staging Iceberg Table

In [95]:
# Drop flights table
spark.sql('DROP TABLE airline.db.flights PURGE')

DataFrame[]

In [96]:
# Create flights table if not exists
flights_ddl = f"""
CREATE TABLE IF NOT EXISTS airline.db.flights (
    date DATE,
    airline STRING,
    flight_number INT,
    tail_number STRING,
    origin_airport STRING,
    destination_airport STRING,
    scheduled_departure INT,
    departure_time INT,
    departure_delay INT,
    taxi_out INT,
    wheels_off INT,
    scheduled_time INT,
    elapsed_time INT,
    air_time INT,
    distance INT,
    wheels_on INT,
    taxi_in INT,
    scheduled_arrival INT,
    arrival_time INT,
    arrival_delay INT,
    diverted INT,
    cancelled INT,
    cancellation_reason STRING,
    air_system_delay INT,
    security_delay INT,
    airline_delay INT,
    late_aircraft_delay INT,
    weather_delay INT,
    is_delayed INT
)
USING iceberg
PARTITIONED BY (month(date))
"""

spark.sql(flights_ddl)

DataFrame[]

### Basic write implementation

In [97]:
# Write flights data to table
flights_df \
    .writeTo('airline.db.flights') \
    .append()

### Idempotent write implementation

In [21]:
# IDEMPOTENT

# Create temp view
flights_df.createOrReplaceTempView('flights')

# Write flights data to table
flights_merge_ddl = """
MERGE INTO airline.db.flights t
USING (SELECT * FROM flights) s
    ON  t.date = s.date
    AND t.flight_number = s.flight_number
WHEN NOT MATCHED THEN INSERT *
"""

# Double checking
spark.sql('SELECT COUNT(*) FROM airline.db.flights').show()

# spark.sql(flights_merge_ddl)

# Double checking
spark.sql('SELECT COUNT(*) FROM airline.db.flights').show()

# Check snapshot history and manifests for changes
snapshots = spark.sql('SELECT * FROM airline.db.flights.snapshots').collect()
snapshots = [snap.asDict() for snap in snapshots]
import pprint as pp
pp.pp(snapshots)

+--------+
|count(1)|
+--------+
| 5819079|
+--------+

+--------+
|count(1)|
+--------+
| 5819079|
+--------+

[{'committed_at': datetime.datetime(2024, 12, 22, 4, 5, 47, 694000),
  'snapshot_id': 6483349522153703138,
  'parent_id': None,
  'operation': 'append',
  'manifest_list': 's3://warehouse/db/flights/metadata/snap-6483349522153703138-1-29cc0a41-9f91-41a9-a3a9-8594bcfec6ed.avro',
  'summary': {'spark.app.id': 'local-1734811599173',
              'changed-partition-count': '12',
              'added-data-files': '12',
              'total-equality-deletes': '0',
              'added-records': '5819079',
              'total-position-deletes': '0',
              'added-files-size': '130797157',
              'total-delete-files': '0',
              'total-files-size': '130797157',
              'total-records': '5819079',
              'total-data-files': '12'}},
 {'committed_at': datetime.datetime(2024, 12, 26, 17, 54, 23, 601000),
  'snapshot_id': 6628237988227580671,
  'parent

### WAP write implementation

In [39]:
# Verify table properties with pyiceberg
from pyiceberg.catalog import load_catalog
catalog = load_catalog(
    'airline',
    **{
        "uri": "http://rest:8181",
        "s3.endpoint": "http://minio:9000",
        "py-io-impl": "pyiceberg.io.pyarrow.PyArrowFileIO",
        "s3.access-key-id": "admin",
        "s3.secret-access-key": "password",
    }
)
table = catalog.load_table('db.flights')
print(table.properties)

{'owner': 'root', 'write.parquet.compression-codec': 'zstd'}


In [49]:
CATALOG_NAME = 'airline'
DATABASE_NAME = 'db'

# set table property
spark.sql(f'ALTER TABLE {CATALOG_NAME}.{DATABASE_NAME}.flights SET TBLPROPERTIES ("write.wap.enabled"="true")')
print(table.properties)


# Verify if an audit branch is created.
spark.sql(f"select * from {CATALOG_NAME}.{DATABASE_NAME}.flights.refs").show(truncate=False)

# Create audit branch
audit_branch = 'audit_branch_flights'
spark.sql(f'ALTER TABLE {CATALOG_NAME}.{DATABASE_NAME}.flights CREATE BRANCH {audit_branch}')

# Verify if an audit branch is created.
spark.sql(f"select * from {CATALOG_NAME}.{DATABASE_NAME}.flights.refs").show(truncate=False)



{'owner': 'root', 'write.parquet.compression-codec': 'zstd'}
+--------------------+------+-------------------+-----------------------+---------------------+----------------------+
|name                |type  |snapshot_id        |max_reference_age_in_ms|min_snapshots_to_keep|max_snapshot_age_in_ms|
+--------------------+------+-------------------+-----------------------+---------------------+----------------------+
|main                |BRANCH|8777543075953159173|NULL                   |NULL                 |NULL                  |
|audit_branch_flights|BRANCH|8777543075953159173|NULL                   |NULL                 |NULL                  |
+--------------------+------+-------------------+-----------------------+---------------------+----------------------+



IllegalArgumentException: Ref audit_branch_flights already exists

In [41]:
# Set spark WAP branch in session config
spark.conf.set('spark.wap.branch', audit_branch)
spark.conf.get('spark.wap.branch')

'audit_branch_flights'

In [42]:
spark.sql("""
DELETE FROM airline.db.flights
WHERE date = "2015-12-01"
""")

spark.sql('SELECT COUNT(*) FROM airline.db.flights').show()

+--------+
|count(1)|
+--------+
| 5803256|
+--------+



In [47]:
spark.sql('SELECT * FROM airline.db.flights WHERE date = "2015-12-01"').show()
spark.sql('SELECT * FROM airline.db.flights VERSION AS OF "main" WHERE date = "2015-12-01"').show()

+----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+----------+
|      date|airline|flight_number|tail_number|origin_airport|destination_airport|scheduled_departure|departure_time|departure_delay|taxi_out|wheels_off|scheduled_time|elapsed_time|air_time|distance|wheels_on|taxi_in|scheduled_arrival|arrival_time|arrival_delay|diverted|cancelled|cancellation_reason|air_system_delay|security_delay|airline_delay|late_aircraft_delay|weather_delay|is_delayed|
+----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+

In [44]:
# Perform idempotent write
# Create temp view
flights_df = spark.sql('SELECT * FROM airline.db.flights VERSION AS OF "main"').filter(col('date') == '2015-12-01')
flights_df.explain()
print(flights_df.count())
flights_df.cache()
flights_df.createOrReplaceTempView('flights')

# TODO dedup input from source

# Write flights data to table
flights_merge_ddl = """
MERGE INTO airline.db.flights t
USING (SELECT * FROM flights) s
    ON  t.date = s.date
    AND t.airline = s.airline
    AND t.flight_number = s.flight_number
    AND t.scheduled_departure = s.scheduled_departure
WHEN NOT MATCHED THEN INSERT *
"""

spark.sql(flights_merge_ddl).explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- InMemoryTableScan [date#11549, airline#11550, flight_number#11551, tail_number#11552, origin_airport#11553, destination_airport#11554, scheduled_departure#11555, departure_time#11556, departure_delay#11557, taxi_out#11558, wheels_off#11559, scheduled_time#11560, elapsed_time#11561, air_time#11562, distance#11563, wheels_on#11564, taxi_in#11565, scheduled_arrival#11566, arrival_time#11567, arrival_delay#11568, diverted#11569, cancelled#11570, cancellation_reason#11571, air_system_delay#11572, ... 5 more fields]
      +- InMemoryRelation [date#11549, airline#11550, flight_number#11551, tail_number#11552, origin_airport#11553, destination_airport#11554, scheduled_departure#11555, departure_time#11556, departure_delay#11557, taxi_out#11558, wheels_off#11559, scheduled_time#11560, elapsed_time#11561, air_time#11562, distance#11563, wheels_on#11564, taxi_in#11565, scheduled_arrival#11566, arrival_time#11567, arrival_delay#11568, dive

24/12/28 03:33:20 WARN CacheManager: Asked to cache already cached data.


== Physical Plan ==
CommandResult <empty>
   +- AppendData org.apache.spark.sql.execution.datasources.v2.DataSourceV2Strategy$$Lambda$5024/0x0000000841c6c040@7d0a3304, IcebergWrite(table=airline.db.flights, format=PARQUET)
      +- AdaptiveSparkPlan isFinalPlan=true
         +- == Final Plan ==
            AQEShuffleRead coalesced
            +- ShuffleQueryStage 3
               +- Exchange hashpartitioning(staticinvoke(class org.apache.iceberg.spark.functions.MonthsFunction$DateToMonthsFunction, IntegerType, invoke, date#11549, DateType, false, true, true), 200), REBALANCE_PARTITIONS_BY_COL, 402653184, [plan_id=1270]
                  +- *(4) SortMergeJoin [date#11549, flight_number#11551], [date#12806, flight_number#12808], LeftAnti
                     :- *(2) Sort [date#11549 ASC NULLS FIRST, flight_number#11551 ASC NULLS FIRST], false, 0
                     :  +- AQEShuffleRead coalesced
                     :     +- ShuffleQueryStage 2
                     :        +- Exchange 

In [ ]:
# Perform Data Quality checks



In [45]:
# If all checks pass, cherry-pick snapshot, ie. publish staging branch to main

# Get snapshot id
spark.sql('SELECT snapshot_id FROM airline.db.flights.refs WHERE name="audit_branch_flights"').show()

if True:
    spark.sql(f'CALL airline.system.fast_forward("db.flights", "main", "{audit_branch}")')
    # spark.sql('CALL airline.system.cherrypick_snapshot("db.flights", 668148544964107792)')

+-------------------+
|        snapshot_id|
+-------------------+
|8777543075953159173|
+-------------------+



In [36]:
# Cleanup
# Unset from SparkSession
spark.conf.unset('spark.wap.branch')

# unset from table properties so that create table ddl can be seen again on Athena.
spark.sql(f"ALTER TABLE airline.db.flights UNSET TBLPROPERTIES ('write.wap.enabled')")

# Drop audit branch
spark.sql(f"ALTER TABLE airline.db.flights DROP BRANCH {audit_branch}")

DataFrame[]

In [53]:
spark.sql('SELECT * FROM airline.db.flights.history').show()
spark.sql('SELECT * FROM airline.db.flights.snapshots').show(truncate=False)
spark.sql('SELECT * FROM airline.db.flights.refs').show()

+--------------------+-------------------+-------------------+-------------------+
|     made_current_at|        snapshot_id|          parent_id|is_current_ancestor|
+--------------------+-------------------+-------------------+-------------------+
|2024-12-22 04:05:...|6483349522153703138|               NULL|               true|
|2024-12-26 17:54:...|6628237988227580671|6483349522153703138|               true|
|2024-12-28 00:53:...|7078291234605864939|6628237988227580671|               true|
|2024-12-28 03:34:...|8777543075953159173|5764227798246680375|               true|
+--------------------+-------------------+-------------------+-------------------+

+-----------------------+-------------------+-------------------+---------+-------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------

In [38]:
spark.sql('SELECT * FROM airline.db.flights WHERE date="2015-12-01"').show()

+----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+----------+
|      date|airline|flight_number|tail_number|origin_airport|destination_airport|scheduled_departure|departure_time|departure_delay|taxi_out|wheels_off|scheduled_time|elapsed_time|air_time|distance|wheels_on|taxi_in|scheduled_arrival|arrival_time|arrival_delay|diverted|cancelled|cancellation_reason|air_system_delay|security_delay|airline_delay|late_aircraft_delay|weather_delay|is_delayed|
+----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+

In [ ]:
# # Create flights table if not exists
# spark.sql("""
# CREATE TABLE IF NOT EXISTS airline.db.flights (
#     year INT,
#     month INT,
#     day INT,
#     day_of_week INT,
#     airline STRING,
#     flight_number INT,
#     tail_number STRING,
#     origin_airport STRING,
#     destination_airport STRING,
#     scheduled_departure INT,
#     departure_time INT,
#     departure_delay INT,
#     taxi_out INT,
#     wheels_off INT,
#     scheduled_time INT,
#     elapsed_time INT,
#     air_time INT,
#     distance INT,
#     wheels_on INT,
#     taxi_in INT,
#     scheduled_arrival INT,
#     arrival_time INT,
#     arrival_delay INT,
#     diverted INT,
#     cancelled INT,
#     cancellation_reason STRING,
#     air_system_delay INT,
#     security_delay INT,
#     airline_delay INT,
#     late_aircraft_delay INT,
#     weather_delay INT
# )
# USING iceberg
# PARTITIONED BY (month)
# """)

In [24]:
# # Create database if not exists
# spark.sql('CREATE DATABASE IF NOT EXISTS db')

# print(spark.catalog.tableExists('airline.db.flights'))

# # Create flights table if not exists
# if not spark.catalog.tableExists('airline.db.flights'):
#     spark.catalog.createTable(
#         'airline.db.flights',
#         schema=flights_schema,
#         source='iceberg',
#         partitioning='month'
#     )

# spark.catalog.getTable('airline.db.flights')

False


Table(name='flights', catalog='airline', namespace=['db'], description=None, tableType='MANAGED', isTemporary=False)

#### Checking table metadata

In [99]:
spark.sql('DESCRIBE EXTENDED airline.db.flights').collect()

[Row(col_name='date', data_type='date', comment=None),
 Row(col_name='airline', data_type='string', comment=None),
 Row(col_name='flight_number', data_type='int', comment=None),
 Row(col_name='tail_number', data_type='string', comment=None),
 Row(col_name='origin_airport', data_type='string', comment=None),
 Row(col_name='destination_airport', data_type='string', comment=None),
 Row(col_name='scheduled_departure', data_type='int', comment=None),
 Row(col_name='departure_time', data_type='int', comment=None),
 Row(col_name='departure_delay', data_type='int', comment=None),
 Row(col_name='taxi_out', data_type='int', comment=None),
 Row(col_name='wheels_off', data_type='int', comment=None),
 Row(col_name='scheduled_time', data_type='int', comment=None),
 Row(col_name='elapsed_time', data_type='int', comment=None),
 Row(col_name='air_time', data_type='int', comment=None),
 Row(col_name='distance', data_type='int', comment=None),
 Row(col_name='wheels_on', data_type='int', comment=None),
 R

In [143]:
spark.sql('SHOW CREATE TABLE airline.db.flights').collect()

[Row(createtab_stmt="CREATE TABLE airline.db.flights (\n  year INT,\n  month INT,\n  day INT,\n  day_of_week INT,\n  airline STRING,\n  flight_number INT,\n  tail_number STRING,\n  origin_airport STRING,\n  destination_airport STRING,\n  scheduled_departure INT,\n  departure_time INT,\n  departure_delay INT,\n  taxi_out INT,\n  wheels_off INT,\n  scheduled_time INT,\n  elapsed_time INT,\n  air_time INT,\n  distance INT,\n  wheels_on INT,\n  taxi_in INT,\n  scheduled_arrival INT,\n  arrival_time INT,\n  arrival_delay INT,\n  diverted INT,\n  cancelled INT,\n  cancellation_reason STRING,\n  air_system_delay INT,\n  security_delay INT,\n  airline_delay INT,\n  late_aircraft_delay INT,\n  weather_delay INT)\nUSING iceberg\nPARTITIONED BY (month)\nLOCATION 's3://warehouse/db/flights'\nTBLPROPERTIES (\n  'current-snapshot-id' = '6887883028741510457',\n  'format' = 'iceberg/parquet',\n  'format-version' = '2',\n  'write.parquet.compression-codec' = 'zstd')\n")]

In [8]:
metrics = spark.sql('select readable_metrics from airline.db.flights.files').collect()

# Check lower and upper bounds on month to ensure partitioning by month is being performed correctly
for metric in metrics:
    temp = metric.asDict()['readable_metrics'].asDict()['month']
    print(temp)

KeyError: 'month'

In [100]:
%%sql
SELECT
    *
FROM airline.db.flights.partitions

24/12/22 04:07:19 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


partition,spec_id,record_count,file_count,total_data_file_size_in_bytes,position_delete_record_count,position_delete_file_count,equality_delete_record_count,equality_delete_file_count,last_updated_at,last_updated_snapshot_id
Row(date_month=550),0,467972,1,10431197,0,0,0,0,2024-12-22 04:05:47.694000,6483349522153703138
Row(date_month=551),0,479230,1,10826942,0,0,0,0,2024-12-22 04:05:47.694000,6483349522153703138
Row(date_month=540),0,469968,1,10589549,0,0,0,0,2024-12-22 04:05:47.694000,6483349522153703138
Row(date_month=541),0,429191,1,9753720,0,0,0,0,2024-12-22 04:05:47.694000,6483349522153703138
Row(date_month=544),0,496993,1,11149328,0,0,0,0,2024-12-22 04:05:47.694000,6483349522153703138
Row(date_month=545),0,503897,1,11497726,0,0,0,0,2024-12-22 04:05:47.694000,6483349522153703138
Row(date_month=542),0,504312,1,11364375,0,0,0,0,2024-12-22 04:05:47.694000,6483349522153703138
Row(date_month=543),0,485151,1,10875997,0,0,0,0,2024-12-22 04:05:47.694000,6483349522153703138
Row(date_month=548),0,464946,1,10274254,0,0,0,0,2024-12-22 04:05:47.694000,6483349522153703138
Row(date_month=549),0,486165,1,10710799,0,0,0,0,2024-12-22 04:05:47.694000,6483349522153703138


In [32]:
%%sql
SELECT
    *
FROM airline.db.flights.files

+---------+--------------------------------------------------------------------------------------------------------+-------------+---------+---------------+--------------+--------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------------+---------------+--------------+---------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [101]:
%%sql
SELECT
    SUM(file_size_in_bytes)
FROM airline.db.flights.files

sum(file_size_in_bytes)
130797157


In [19]:
%%sql
SELECT 
    partition
FROM 
    airline.db.flights.files
WHERE 
    null_value_counts['7'] > 0
GROUP BY 
    partition

partition
Row(month=12)
Row(month=1)
Row(month=6)
Row(month=3)
Row(month=5)
Row(month=9)
Row(month=4)
Row(month=8)
Row(month=7)
Row(month=10)


# Perform Agg_fact_flights transfrom

In [3]:
# Read flights table
flights_df = spark.table('airline.db.flights')

# Join dim_date table to flights
flights_df = flights_df \
    .join(
        broadcast(date_df),
        ['date']
    )

flights_df.createOrReplaceTempView('flights')

flights_df.show()
# flights_df.explain()

24/12/26 17:45:42 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+----------+----+-----+---+-----------+-------+------------+----------+
|      date|airline|flight_number|tail_number|origin_airport|destination_airport|scheduled_departure|departure_time|departure_delay|taxi_out|wheels_off|scheduled_time|elapsed_time|air_time|distance|wheels_on|taxi_in|scheduled_arrival|arrival_time|arrival_delay|diverted|cancelled|cancellation_reason|air_system_delay|security_delay|airline_delay|late_aircraft_delay|weather_delay|is_delayed|year|month|day|day_of_week|quarter|holiday_name|is_holiday|
+----------+-------+-------------+-----------+--------------+-------------------+-----------------

In [5]:
def agg_delay_metrics_rollup(
    input_df: DataFrame,
    non_time_columns: list[str],
    time_columns: list[str]
) -> DataFrame:

    # Concatenate both list of columns
    cube_columns = non_time_columns + time_columns

    # Calculate the aggregate values for the cube
    output_df = input_df \
        .rollup(*cube_columns) \
        .agg(
            count('*').alias('total_flights'),
            count(when(col('is_delayed') == True, 1)).alias('delayed_flights'),
            ( count(when(col('is_delayed') == True, 1)) / count('*') ).alias('delay_rate'),
            avg('departure_delay').alias('avg_delay_time')
        )

    # Calculate aggregation label to add as column
    time_agg_level = get_aggregation_level(time_columns)
    agg_level = get_aggregation_level(non_time_columns)

    # Add (time_)agg_level columns
    output_df = output_df \
        .withColumn('time_agg_level', time_agg_level) \
        .withColumn('agg_level', agg_level)
    
    return output_df

def get_aggregation_level(columns: list[str]):
    # TODO add comments
    
    # 
    # Returns col
    agg_level = concat_ws(
        '_',
        *[when(col(c).isNotNull(), lit(c)) for c in columns]
    )

    all_nulls = lit(True)
    for c in columns:
        all_nulls &= col(c).isNull()
    agg_level = when(all_nulls, 'all').otherwise(agg_level)
    
    return agg_level

# cols = ['month', 'day', 'airport']
# print(get_aggregation_level(cols))


# Full Outer Join cube by cancelled, times
# Dedup the "all" agg_levels
# agg_df.sort('time_agg_level', 'agg_level').show(100)

In [162]:
time_columns = ['year', 'month']
non_time_columns = ['airline', 'origin_airport']
temp_df = flights_df.sample(0.1)

agg_df = agg_delay_metrics_cube_by_cols(temp_df, time_columns, non_time_columns)
print(agg_df.count())
agg_df.filter(col('time_agg_level') == lit('year_month')).show(100)

39852


+----+-----+-------+--------------+-------------+---------------+--------------------+-------------------+--------------+--------------------+
|year|month|airline|origin_airport|total_flights|delayed_flights|          delay_rate|     avg_delay_time|time_agg_level|           agg_level|
+----+-----+-------+--------------+-------------+---------------+--------------------+-------------------+--------------+--------------------+
|2015|    7|     WN|           MCO|          369|            183|  0.4959349593495935| 18.489071038251367|    year_month|airline_origin_ai...|
|2015|    7|     WN|           SEA|          145|             79|  0.5448275862068965| 10.393103448275863|    year_month|airline_origin_ai...|
|2015|    7|     AA|           FLL|           89|             23| 0.25842696629213485| 2.8764044943820224|    year_month|airline_origin_ai...|
|2015|    7|     UA|           OGG|           26|             12| 0.46153846153846156| 2.4615384615384617|    year_month|airline_origin_ai...|

In [201]:
# Confirm redundant aggs for cube with cancelled and cancellation_reason
time_columns = ['year', 'month']
non_time_columns = ['airline', 'origin_airport', 'cancelled', 'cancellation_reason']
agg_df = agg_delay_metrics_cube_by_cols(temp_df, time_columns, non_time_columns)

# agg_df.filter(col('agg_level') == 'cancelled_cancellation_reason')).show(100)
agg_df \
    .filter(
        (col('year').isNull()) &
        (col('month').isNull()) &
        (col('airline').isNull()) &
        (col('origin_airport').isNull()) &
        (col('cancellation_reason') == lit('A'))
    ) \
    .show()

+----+-----+-------+--------------+---------+-------------------+-------------+---------------+-------------------+-----------------+--------------+--------------------+
|year|month|airline|origin_airport|cancelled|cancellation_reason|total_flights|delayed_flights|         delay_rate|   avg_delay_time|time_agg_level|           agg_level|
+----+-----+-------+--------------+---------+-------------------+-------------+---------------+-------------------+-----------------+--------------+--------------------+
|NULL| NULL|   NULL|          NULL|        1|                  A|         2375|             86|0.03621052631578947|66.87837837837837|           all|cancelled_cancell...|
|NULL| NULL|   NULL|          NULL|     NULL|                  A|         2375|             86|0.03621052631578947|66.87837837837837|           all| cancellation_reason|
+----+-----+-------+--------------+---------+-------------------+-------------+---------------+-------------------+-----------------+--------------+--

In [203]:
cols = ['cancellation_reason', 'year', 'month', 'day_of_week', 'airline']
flights_df.rollup(*cols) \
    .agg(
        count('*').alias('total_flights'),
        count(when(col('is_delayed') == True, 1)).alias('delayed_flights'),
        ( count(when(col('is_delayed') == True, 1)) / count('*') ).alias('delay_rate'),
        avg('departure_delay').alias('avg_delay_time')
    ) \
    .show()

+-------------------+----+-----+-------+-------------+---------------+--------------------+------------------+
|cancellation_reason|year|month|airline|total_flights|delayed_flights|          delay_rate|    avg_delay_time|
+-------------------+----+-----+-------+-------------+---------------+--------------------+------------------+
|                  A|NULL| NULL|   NULL|        25262|            991| 0.03922888132372734| 57.02896725440806|
|                  A|2015| NULL|   NULL|        25262|            991| 0.03922888132372734| 57.02896725440806|
|                  A|2015|    7|     F9|           29|              1|0.034482758620689655|-4.333333333333333|
|               NULL|NULL| NULL|   NULL|      5729195|        2123078|  0.3705717818995513| 9.338837306113685|
|               NULL|2015|    7|     DL|        80635|          27722|  0.3437961183109072| 8.267278477088112|
|                  C|2015|    7|     NK|           19|              2| 0.10526315789473684|              84.0|
|

In [7]:
cols = ['tail_number', 'year', 'month']
flights_df.rollup(*cols) \
    .agg(
        count('*').alias('total_flights'),
        count(when(col('is_delayed') == True, 1)).alias('delayed_flights'),
        ( count(when(col('is_delayed') == True, 1)) / count('*') ).alias('delay_rate'),
        avg('departure_delay').alias('avg_delay_time')
    ) \
    .filter(col('year').isNull()) \
    .show()

+-----------+----+-----+-------------+---------------+-------------------+-------------------+
|tail_number|year|month|total_flights|delayed_flights|         delay_rate|     avg_delay_time|
+-----------+----+-----+-------------+---------------+-------------------+-------------------+
|     N821DN|NULL| NULL|         1311|            488|0.37223493516399697|  6.160796324655436|
|     N997DL|NULL| NULL|         1495|            477|0.31906354515050167|  7.437121721587088|
|     N209WN|NULL| NULL|         1927|            849| 0.4405812143227815|   9.75614861329147|
|     N454AA|NULL| NULL|          914|            293|0.32056892778993434|  7.565759637188209|
|     N506CA|NULL| NULL|         2027|            453|0.22348297977306364| 3.3966118584952665|
|     N510UW|NULL| NULL|         1275|            483| 0.3788235294117647|  8.430588235294117|
|     N519UW|NULL| NULL|         1075|            413| 0.3841860465116279|  8.950697674418604|
|     N230WN|NULL| NULL|         1908|            

In [16]:
cols = ['airline', 'year', 'month']
flights_df.rollup(*cols) \
    .agg(
        count('*').alias('total_flights'),
        count(when(col('is_delayed') == True, 1)).alias('delayed_flights'),
        ( count(when(col('is_delayed') == True, 1)) / count('*') ).alias('delay_rate'),
        avg('departure_delay').alias('avg_delay_time')
    ) \
    .filter(col('year').isNull()) \
    .show()

+-------+----+-----+-------------+---------------+-------------------+-------------------+
|airline|year|month|total_flights|delayed_flights|         delay_rate|     avg_delay_time|
+-------+----+-----+-------------+---------------+-------------------+-------------------+
|     F9|NULL| NULL|        90836|          34893| 0.3841318420009688| 13.350858345331709|
|   NULL|NULL| NULL|      5819079|        2125618| 0.3652842657747042|  9.370158275198389|
|     DL|NULL| NULL|       875881|         282463|0.32249015562616384|   7.36925417661782|
|     NK|NULL| NULL|       117379|          52089| 0.4437676245324973| 15.944765880783688|
|     AS|NULL| NULL|       172521|          43566|0.25252577947032534| 1.7858007096736666|
|     UA|NULL| NULL|       515723|         256550|0.49745696817865404| 14.435441010805953|
|     AA|NULL| NULL|       725984|         245904| 0.3387182086657557|  8.900856346719806|
|     WN|NULL| NULL|      1261855|         566807|0.44918552448577687| 10.581986295158847|

In [ ]:
# Individual rollups

# 1
time_columns = ['year', 'month', 'day_of_week']
non_time_columns = ['airline', 'origin_airport', 'cancelled', 'cancellation_reason']

agg_df = agg_delay_metrics_cube_by_cols(flights_df, time_columns, non_time_columns)


# 2


In [50]:
grouping_set_sql = """
SELECT
    airline,
    year,
    month,
    day_of_week,
    COUNT(*) AS total_flights,
    COUNT(CASE WHEN is_delayed = 1 THEN 1 END) AS delayed_flights,
    delayed_flights / total_flights AS delay_rate,
    AVG(departure_delay) AS avg_delay_time
FROM flights
GROUP BY
    GROUPING SETS (
        
            --airline,
            --day_of_week,
            --ROLLUP(year, month),
            --GROUPING SETS(airline, ROLLUP(year, month))
            CUBE(airline, (year, month), day_of_week)
        
    )
"""


temp_df = spark.sql(grouping_set_sql)

# temp_df.show()

print('==========================')
print('day_of_week and NULL month')
print('==========================')
temp_df.filter((col('day_of_week').isNotNull()) & (col('month').isNull())).show()
temp_df.filter((col('day_of_week').isNotNull()) & (col('airline').isNull())).show()

print('==========================')
print('day_of_week and NULL month')
print('==========================')
temp_df.filter((col('year').isNotNull()) & (col('month').isNull())).show()

day_of_week and NULL month
+-------+----+-----+-----------+-------------+---------------+-------------------+-------------------+
|airline|year|month|day_of_week|total_flights|delayed_flights|         delay_rate|     avg_delay_time|
+-------+----+-----+-----------+-------------+---------------+-------------------+-------------------+
|     NK|NULL| NULL|          7|        16844|           7631| 0.4530396580384707| 15.780295269659536|
|     HA|NULL| NULL|          1|        10867|           2846|0.26189380693843745|0.13823908638791674|
|     UA|NULL| NULL|          6|        77634|          39820|  0.512919597083752| 13.672101590266427|
|     F9|NULL| NULL|          6|        13242|           5157| 0.3894426823742637| 12.633724895793861|
|     WN|NULL| NULL|          6|       185731|          88275| 0.4752841475036477| 11.120365505549206|
|     UA|NULL| NULL|          5|        79126|          40928| 0.5172509668124258|  14.88742760925581|
|     AA|NULL| NULL|          6|       106374|

In [207]:
# Count number of rows in full cube (combinatorial explosion)
time_columns = ['year', 'month', 'day_of_week']
non_time_columns = ['airline', 'origin_airport', 'cancelled', 'cancellation_reason']

agg_df = agg_delay_metrics_cube_by_cols(flights_df, time_columns, non_time_columns)

agg_df.count()

24/12/23 04:54:25 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/23 04:54:30 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/23 04:54:35 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/23 04:54:44 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/23 04:54:48 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/23 04:54:55 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/23 04:54:57 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/23 04:55:01 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
24/12/23 04:55:02 WARN RowBasedKeyValueBatch: Calling spill() on

1814436

In [211]:
# Given columns
# columns = [
#     "year", "month", "day_of_week", "airline", "origin_airport", "tail_number",
#     "cancelled", "cancellation_reason"
# ]

columns = [
    "year", "airline", "origin_airport", "tail_number", "cancelled", "cancellation_reason"
]

# Generate all non-empty combinations
all_combinations = []
for r in range(1, len(columns) + 1):  # r ranges from 1 to len(columns)
    all_combinations.extend(itertools.combinations(columns, r))

# Print all combinations
for combination in all_combinations:
    print(combination)

('year',)
('airline',)
('origin_airport',)
('tail_number',)
('cancelled',)
('cancellation_reason',)
('year', 'airline')
('year', 'origin_airport')
('year', 'tail_number')
('year', 'cancelled')
('year', 'cancellation_reason')
('airline', 'origin_airport')
('airline', 'tail_number')
('airline', 'cancelled')
('airline', 'cancellation_reason')
('origin_airport', 'tail_number')
('origin_airport', 'cancelled')
('origin_airport', 'cancellation_reason')
('tail_number', 'cancelled')
('tail_number', 'cancellation_reason')
('cancelled', 'cancellation_reason')
('year', 'airline', 'origin_airport')
('year', 'airline', 'tail_number')
('year', 'airline', 'cancelled')
('year', 'airline', 'cancellation_reason')
('year', 'origin_airport', 'tail_number')
('year', 'origin_airport', 'cancelled')
('year', 'origin_airport', 'cancellation_reason')
('year', 'tail_number', 'cancelled')
('year', 'tail_number', 'cancellation_reason')
('year', 'cancelled', 'cancellation_reason')
('airline', 'origin_airport', 'tail

### Lets manually code the following:
1. % delay rate of all flights
2. flight volume by month, day of week, month_airline, etc.

In [110]:
# 1. % delay rate of all flights
flights_df \
    .groupBy('airline') \
    .agg(
        count('*').alias('total_flights'),
        count(when(col('is_delayed') == True, 1)).alias('delayed_flights'),
        ( count(when(col('is_delayed') == True, 1)) / count('*') ).alias('delay_rate')
    ) \
    .show()

+-------+-------------+---------------+-------------------+
|airline|total_flights|delayed_flights|         delay_rate|
+-------+-------------+---------------+-------------------+
|     UA|       515723|         256550|0.49745696817865404|
|     NK|       117379|          52089| 0.4437676245324973|
|     AA|       725984|         245904| 0.3387182086657557|
|     EV|       571977|         169897| 0.2970346709745322|
|     B6|       267048|         102061| 0.3821822294119409|
|     DL|       875881|         282463|0.32249015562616384|
|     OO|       588353|         171572| 0.2916140480289894|
|     F9|        90836|          34893| 0.3841318420009688|
|     MQ|       294632|          93726| 0.3181120855847294|
|     HA|        76272|          20146| 0.2641336270190896|
|     AS|       172521|          43566|0.25252577947032534|
|     VX|        61903|          23379| 0.3776715183432144|
|     WN|      1261855|         566807|0.44918552448577687|
|     US|       198715|          62565|0

In [113]:
# 2. flight volume
flights_df \
    .cube('month', 'airline') \
    .agg(
        count('*').alias('total_flights')
    ).show()

+-----+-------+-------------+
|month|airline|total_flights|
+-----+-------+-------------+
| NULL|     B6|       267048|
| NULL|     NK|       117379|
|    7|     HA|         6955|
|    7|     F9|         8090|
| NULL|     HA|        76272|
|    7|     VX|         5411|
| NULL|     VX|        61903|
|    7|     NK|        10351|
|    7|     B6|        24029|
|    7|     MQ|        24750|
|    7|     AA|        81434|
|    7|   NULL|       520718|
|    7|     OO|        52627|
| NULL|     DL|       875881|
| NULL|   NULL|      5819079|
| NULL|     WN|      1261855|
|    7|     WN|       113650|
| NULL|     F9|        90836|
| NULL|     OO|       588353|
| NULL|     MQ|       294632|
+-----+-------+-------------+
only showing top 20 rows



In [82]:
flights_df.filter(col('departure_delay') > 0).groupBy('airline').agg(count('*')).show()

+-------+--------+
|airline|count(1)|
+-------+--------+
|     UA|  256550|
|     NK|   52089|
|     AA|  245904|
|     EV|  169897|
|     B6|  102061|
|     DL|  282463|
|     OO|  171572|
|     F9|   34893|
|     MQ|   93726|
|     HA|   20146|
|     AS|   43566|
|     VX|   23379|
|     WN|  566807|
|     US|   62565|
+-------+--------+

